In [1]:
import pickle
import pandas as pd
import sklearn
import numpy as np
import scipy
# load the dic file

from utils import *

path = 'cleaned/module_dict.pickle'
with open(path, 'rb') as f:
    module_dict = pickle.load(f)

In [2]:
tissues = ['islet', 'liver', 'adipose', 'kidney', 'gastroc']
signs = ['SIGNED', 'UNSIGNED']

In [3]:
X_train, X_test, Y_train, Y_test = get_data('islet','SIGNED','red')

In [4]:
# print the shape of training / testing
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((441, 764), (50, 764), (441, 4), (50, 4))

In [5]:
# use sklearn knn regression method to train a model then predict
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

# train a model
rds = np.random.RandomState(0)
model = Lasso(random_state=rds,selection='random',max_iter=3000,tol=0.0005)
model.get_params()

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': 3000,
 'normalize': 'deprecated',
 'positive': False,
 'precompute': False,
 'random_state': RandomState(MT19937) at 0x207C04CC6A8,
 'selection': 'random',
 'tol': 0.0005,
 'warm_start': False}

In [6]:
alpha= [0.1,1,10,100,1000,5000,10000]
param_grid = {"alpha": alpha}
gsh = GridSearchCV(estimator=model, param_grid=param_grid)
gsh.fit(X_train, Y_train)
# predict
Y_pred = gsh.predict(X_test)

In [7]:
# visualize the gsh.cv_results_
gsh.cv_results_

{'mean_fit_time': array([0.07909651, 0.0211587 , 0.01650391, 0.01681495, 0.01720343,
        0.01670213, 0.01470771]),
 'std_fit_time': array([0.01881996, 0.00254459, 0.00126798, 0.00146672, 0.00147297,
        0.00107518, 0.00124481]),
 'mean_score_time': array([0.00540476, 0.00580063, 0.00540886, 0.00601039, 0.00530562,
        0.00600591, 0.00620413]),
 'std_score_time': array([0.00101931, 0.00074761, 0.00102159, 0.00089606, 0.00074744,
        0.00063046, 0.00147383]),
 'param_alpha': masked_array(data=[0.1, 1, 10, 100, 1000, 5000, 10000],
              mask=[False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'alpha': 0.1},
  {'alpha': 1},
  {'alpha': 10},
  {'alpha': 100},
  {'alpha': 1000},
  {'alpha': 5000},
  {'alpha': 10000}],
 'split0_test_score': array([ 0.34624739,  0.17083073, -0.02782018, -0.02782018, -0.02782018,
        -0.02782018, -0.02782018]),
 'split1_test_score': array([ 0.43770964,  0.1595113 , -0.04

In [8]:
# compute the average mean percentage error
def mean_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true),axis=0) * 100

per_err = mean_percentage_error(Y_test, Y_pred)
# compute the mean percentage error
per_err_mean = np.mean(per_err,axis=0)
# merge per_err and per_err_mean as pandas dataframe
per_err['mean'] = per_err_mean
per_err

glucose          14.736174
weight            9.627620
insulin         104.858906
triglyceride     33.088744
mean             40.577861
dtype: float64